### Seven bacterial metabolites are uptaked and secreted

In [2]:
import cobra
import pandas as pd
import os
# import gurobipy

In [52]:
# specie_name: specie name
# model_xml: model
# model_type: CarveMe or KBase or KBase_DEMETER or CarveMe_DEMETER
# uptake_secrete: uptake or secrete
# experiment_data: experiment data
def metabolite_uptake(specie_name, model_xml, model_type, uptake_secrete, experiment_data):
    bacdive_uptake = pd.read_csv(experiment_data, sep='\t')
    if specie_name in list(bacdive_uptake['MicrobeID']):
        selected_rows = bacdive_uptake[bacdive_uptake['MicrobeID'] == specie_name]
        selected_rows = selected_rows.T.reset_index()
        selected_rows.columns = ['name', 'Value']
        model = cobra.io.read_sbml_model(model_xml)
        # model.solver = 'gurobi'
        ex = []
        for i in model.exchanges:
            id  = i.id
            ex.append(id)
        
        for i in model.exchanges:
            i.bounds = (-1000, 1000)
            
        tol = 0.0001
        TP_uptake, FN_uptake, FP_uptake, TN_uptake = [], [], [], []
        TP_secrete, FN_secrete, FP_secrete, TN_secrete = [], [], [], []
        
        # model_type
        model_type_dict = {
            'CarveMe': 'EX_{}_e',
            'KBase': 'EX_{}_e0',
            'KBase_DEMETER': 'EX_{}(e)',
            'CarveMe_DEMETER': 'EX_{}(e)'}
        def get_rect(name):
            return model_type_dict.get(model_type, 'EX_{}_e').format(name)
        
        for i,j in selected_rows.iloc():
            rect = get_rect(i)
            
            #### uptake
            if uptake_secrete == 'uptake':
                if j == 1:
                    if rect in ex:
                        model.objective = rect
                        FBA = model.optimize('minimize')
                        if FBA.objective_value < -tol:
                            TP_uptake.append(i)
                        else:
                            FN_uptake.append(i)
                    else:
                        FN_uptake.append(i)
                elif j == -1:
                    if rect in ex:
                        model.objective = rect
                        FBA = model.optimize('minimize')
                        if FBA.objective_value < -tol:
                            FP_uptake.append(i)
                        else:
                            TN_uptake.append(i)
                    else:
                        TN_uptake.append(i)
                        
            #### secrete
            elif uptake_secrete == 'secrete':
                if j == 1:
                    if rect in ex:
                        model.objective = rect
                        FBA = model.optimize('maximize')
                        if FBA.objective_value > tol:
                            TP_secrete.append(i)
                        else:
                            FN_secrete.append(i)
                    else:
                        FN_secrete.append(i)
                elif j == -1:
                    if rect in ex:
                        model.objective = rect
                        FBA = model.optimize('maximize')
                        if FBA.objective_value > tol:
                            FP_secrete.append(i)
                        else:
                            TN_secrete.append(i)
                    else:
                        TN_secrete.append(i)            

        if uptake_secrete == 'uptake':
            accuracy = (len(TP_uptake) + len(TN_uptake)) / (len(TP_uptake) + len(TN_uptake) + len(FN_uptake) + len(FP_uptake))
        elif uptake_secrete == 'secrete':
            accuracy = (len(TP_secrete) + len(TN_secrete)) / (len(TP_secrete) + len(TN_secrete) + len(FN_secrete) + len(FP_secrete))
    else:
        accuracy = 'NA'
    return accuracy

In [1]:
# accuracy = metabolite_uptake(specie_name = 'Coprococcus_catus_GD_7', 
#                   model_xml = '/home/gzy/daixiewangluofangfabijiao/GEM/carveme/Eubacterium_hallii_DSM_3353.xml', 
#                   model_type = 'CarveMe', 
#                   uptake_secrete = 'uptake',
#                   experiment_data = '/home/gzy/daixiewangluofangfabijiao/uptake_secretion/carveme/experiment_data/NJC19_Uptake_Data_carveme.txt')
# print(accuracy)

In [59]:
experiment_data_path = './experiment_data/metabolite_uptake_secrete/KBase_DEMETER/NJC19_Uptake_Data_KBase_DEMETER.txt'
model_Data_path = './model/metabolite_uptake_secrete/KBase_DEMETER/'
for model in os.listdir(model_Data_path):
    model_path = os.path.join(model_Data_path, model)
    model_name = model.split('_kbase_refined.xml')[0]
    accuracy = metabolite_uptake(specie_name = model_name, 
                  model_xml = model_path, 
                  model_type = 'KBase_DEMETER', 
                  uptake_secrete = 'uptake',
                  experiment_data = experiment_data_path)
    print(model_name, ":", accuracy)

/tmp/ipykernel_1386384/3417778669.py:8: DtypeWarning: Columns (237) have mixed types. Specify dtype option on import or set low_memory=False.
  bacdive_uptake = pd.read_csv(experiment_data, sep='\t')
/tmp/ipykernel_1386384/3417778669.py:8: DtypeWarning: Columns (237) have mixed types. Specify dtype option on import or set low_memory=False.
  bacdive_uptake = pd.read_csv(experiment_data, sep='\t')


Clostridium_sp_L2_50 : NA
Faecalibacterium_prausnitzii_SL3_3 : 0.7


/tmp/ipykernel_1386384/3417778669.py:8: DtypeWarning: Columns (237) have mixed types. Specify dtype option on import or set low_memory=False.
  bacdive_uptake = pd.read_csv(experiment_data, sep='\t')


Subdoligranulum_variabile_DSM_15176 : 0.8823529411764706


/tmp/ipykernel_1386384/3417778669.py:8: DtypeWarning: Columns (237) have mixed types. Specify dtype option on import or set low_memory=False.
  bacdive_uptake = pd.read_csv(experiment_data, sep='\t')


...

Roseburia_intestinalis_L1_82 : 0.9047619047619048


/tmp/ipykernel_1386384/3417778669.py:8: DtypeWarning: Columns (237) have mixed types. Specify dtype option on import or set low_memory=False.
  bacdive_uptake = pd.read_csv(experiment_data, sep='\t')


Eubacterium_rectale_M104_1 : 0.8666666666666667


/tmp/ipykernel_1386384/3417778669.py:8: DtypeWarning: Columns (237) have mixed types. Specify dtype option on import or set low_memory=False.
  bacdive_uptake = pd.read_csv(experiment_data, sep='\t')


Eubacterium_hallii_DSM_3353 : 0.9230769230769231


### Prediction accuracy of carbon source utilization in three experimental bacteria

In [25]:
# model_type: KBase_DEMETER_id or research_id
# id_type: BiGG or SEED or KEGG or KBase_DEMETER
def carbon_utilize(model_xml, experiment_data, id_type, model_type):
    model = cobra.io.read_sbml_model(model_xml)
    for i in model.exchanges:
        i.bounds = (-1000,1000)
        
    carbon = pd.read_excel(experiment_data)
    TP, TN, FP, FN = [], [], [], []

    model_type_dict = {
        'BiGG': 'EX_{}_e',
        'SEED': 'EX_{}(e)',
        'KEGG': 'EX_{}_e_',
        'KBase_DEMETER': 'EX_{}(e)'}
    def get_rect(name):
        return model_type_dict.get(id_type, 'EX_{}_e').format(name)
    
    for i, j in zip(carbon['Experiment_Result'], carbon[model_type]):
        # EX = 'EX_' + str(j) + '_e'
        EX = get_rect(str(j))
        if EX in model.reactions:
            model.objective = model.reactions.get_by_id(EX)
            fba = model.optimize('minimize').objective_value
            if fba < 0:
                if str(i) == 'True':
                    TP.append(j)
                else:
                    FP.append(j)
            else:
                if i == 'True':
                    FN.append(j)
                else:
                    TN.append(j)
        else:
            if i == 'True':
                FP.append(j)
            else:
                TN.append(j)
    accuracy = (len(TP) + len(TN)) / (len(TP) + len(TN) + len(FP) + len(FN))
    
    return accuracy

In [30]:
# Salmonella
Salmonella_research_xml = './model/carbon_utilize/STM_v1_0.xml'
Salmonella_research_id_type = 'BiGG'
Salmonella_research_model_type = 'research_id'
Salmonella_xml = './model/carbon_utilize/Salmonella_bongori_N268_08_kbase_refined.xml'
Salmonella_xml_id_type = 'KBase_DEMETER'
Salmonella_xml_model_type = 'KBase_DEMETER_id'
Salmonella_experiment_data = './experiment_data/carbon_utilize/Salmonella_Experiment_Result.xlsx'

accuracy = carbon_utilize(model_xml = Salmonella_xml, 
                          experiment_data = Salmonella_experiment_data, 
                          id_type = 'KBase_DEMETER', 
                          model_type = 'KBase_DEMETER_id')
print('Predicting the accuracy of carbon source utilization:', accuracy)

0.9263157894736842


In [27]:
# Pseudomonas aeruginosa
Pseudomonas_research_xml = './model/carbon_utilize/iPae1146.xml'
Pseudomonas_research_id_type = 'SEED'
Pseudomonas_research_model_type = 'research_id'

Pseudomonas_xml = './model/carbon_utilize/P_aeruginosa_ATCC_9027_kbase_refined_addAD.xml'
Pseudomonas_id_type = 'KBase_DEMETER'
Pseudomonas_model_type = 'KBase_DEMETER_id'

Pseudomonas_experiment_data = './experiment_data/carbon_utilize/Pseudomonas_aeruginosa_Experiment_Result.xlsx'

accuracy = carbon_utilize(model_xml = Pseudomonas_xml, 
                          experiment_data = Pseudomonas_experiment_data, 
                          id_type = 'KBase_DEMETER', 
                          model_type = 'KBase_DEMETER_id')
print('Predicting the accuracy of carbon source utilization:', accuracy)

0.9365079365079365


In [29]:
# Acinetobacter baumannii
Acinetobacter_research_xml = './model/carbon_utilize/iATCC19606_2.xml'
Acinetobacter_research_id_type = 'KEGG'
Acinetobacter_research_model_type = 'research_id'

Acinetobacter_xml = './model/carbon_utilize/Acinetobacter_baumannii_addRXNs.xml'
Acinetobacter_id_type = 'KBase_DEMETER'
Acinetobacter_model_type = 'KBase_DEMETER_id'

Acinetobacter_experiment_data = './experiment_data/carbon_utilize/Acinetobacter_baumannii_Experiment_Result.xlsx'

accuracy = carbon_utilize(model_xml = Acinetobacter_xml, 
                          experiment_data = Acinetobacter_experiment_data, 
                          id_type = 'KBase_DEMETER', 
                          model_type = 'KBase_DEMETER_id')
print('Predicting the accuracy of carbon source utilization:', accuracy)

0.9267015706806283
